# Лабораторная работа №2

In [104]:
import spacy
from tqdm.auto import tqdm
import pandas as pd
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

In [105]:
nlp = spacy.load("ru_core_news_md")

### Работа с данными

In [106]:
"""
Считываем наши данные и оформляем в DataFrame.
"""
df = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding="utf8", sep='\t')
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [107]:
df['sentiment'].value_counts()

sentiment
negative    30000
neautral    30000
positive    30000
Name: count, dtype: int64

In [108]:
"""
Мы оставляем только положительные и отрицательные отзывы.
"""
df = df[df['sentiment'] != 'neautral']
print(df.iloc[0]['sentiment'])
print(df.iloc[0]['review'])


negative
качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......


In [109]:
""" 
Разделяем на обучающую и валидационную выборку.
"""
x_train, x_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size = 0.33, random_state = 42)
x_test

12628    Заказываю юбку второй раз. Первый раз была в в...
67730    Отличные джинсы, почиталась отзывы говорили за...
69991    вполне приличное платье, я довольна покупкой. ...
8525                     Не получила товар. Верните деньги
8279     Ужас. Там где прозрачная ткань при первой стир...
                               ...                        
2715     после стирки блестящая нитка по всей кофте выл...
11309    товар не получила, обещали вернуть деньги, я т...
20807    заказываю 2ой раз 1 раз нормальные а второй ра...
6681     Продавец быстро оформил и выслал товар.Спасибо...
74046    на параметры 105/88/115, рост 168 см подошел X...
Name: review, Length: 19800, dtype: object

In [110]:
""" 
Создаем список кортежей для работы с pipeline.
"""
train = list(zip(x_train, y_train))
train[5]

('После нескольких месяцев ожидания товар так и не получила. Деньги вернули. Не советую покупать у этого продавца.',
 'negative')

In [111]:
valid = list(zip(x_test, y_test))
valid[0]

('Заказываю юбку второй раз. Первый раз была в восторге, но второй раз разочарована полностью. Размер не соответствует, заказывала S с обхватом талии 64, а пришла юбка с обхватом талии 72. Длина также не соответствует заявленной - 69. Пошито криво, сзади одна половинка больше другой, появилась резинка. На первой юбке (мятной) нет резинки. Имеются пятна. Единственное, что порадовало, так это быстрая доставка и полное отслеживание.',
 'negative')

In [112]:
def make_docs(data):
    """
    Преобразует данные в spacy документы

    Параметры:
    ---------
    Список (кортеж (текст, метка))

    Возвращает:
    ----------
    Список (spacy.Doc.doc)
    """
    docs = []
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [113]:
train_docs = make_docs(train)

  0%|          | 0/40200 [00:00<?, ?it/s]

100%|██████████| 40200/40200 [03:41<00:00, 181.60it/s]


In [114]:
valid_docs = make_docs(valid)

100%|██████████| 19800/19800 [01:47<00:00, 183.76it/s]


In [116]:
""" 
Сохраняем наши данные для дальнейшего обучения модели.    
"""
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

### Прописываем с помощью базового конфига config.cfg для обучения модели

In [117]:
! python -m spacy init fill-config base_config.cfg config.cfg

""" 
Созадем конфиг для обучения нашей модели на основе базового ( https://spacy.io/usage/training#quickstart ), указав его имя ( config.cfg ).
"""

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


### Обучение модели

In [119]:
! python -m spacy train config.cfg --output ./output 

""" 
Обучаем нашу модель и прописываем путь, в котором мы сохраняем нашу последнюю и наилучшую модель.
"""

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       33.18    0.33
  0     200         37.60       85.72    0.86
  0     400         24.65       87.32    0.87
  0     600         19.86       88.69    0.89
  0     800         18.58       89.45    0.89
  0    1000         17.54       90.62    0.91
  0    1200         15.29       91.09    0.91
  0    1400         14.99       91.34    0.91
  0    1600         14.12       91.74    0.92
  0    1800         13.32       91.97    0.92
  0    2000         13.43       92.33    0.92
  0    2200         12.37       92.55    0.93
  0    2400         11.95       92.70  

In [ ]:
""" 
Загружаем нашу модель с наилучшим результатом.
"""
nlp = spacy.load("output/model-best")

### Тестирование модели

In [123]:
""" 
Создаем тестовые отзывы и прогоняем их через обученную модель.
"""
text = [ "Отличный товар! Всем домохозяйкам рекомендую." , "Товар просто ужас!!! Никому не рекомендую! Ненавижу вас!"]
for t in text:
    doc = nlp(t)
    print(doc.cats)
    if doc.cats['positive'] > 0.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

{'positive': 0.9271988272666931, 'negative': 0.07280117273330688}
the sentiment is positive
{'positive': 0.07049776613712311, 'negative': 0.9295022487640381}
the sentiment is negative
